# ClockSandbox — Phase I Smoke Test

- Ideal vs Noisy (white fractional-frequency) over 1 day.
- Deterministic seed.
- Plots of readings and difference, plus basic metrics.

In [ ]:
from src.clocks.ideal import IdealClock
from src.clocks.noisy import NoisyOscillatorClock
from src.analysis import run_clocks, compare_clocks, plot_comparison
ideal = IdealClock()
noisy = NoisyOscillatorClock(sigma_y=1e-11, seed=42)
ts = run_clocks([ideal, noisy], duration=86_400.0, dt=1.0)
metrics = compare_clocks(ts["time"], ts["clock_0"], ts["clock_1"])
metrics

In [ ]:
# Plot (disable if running headless)
plot_comparison(ts, labels=["Ideal", "Noisy (white freq)"])


### Overlapping Allan Deviation with Uncertainties (allantools)

This cell uses **allantools** to compute the **overlapping Allan deviation** \(\sigma_y(	au)\)  
including **1-σ uncertainties**, which are suitable for practical stability analysis.

- Function: `adev_overlapping_allantools(y, dt, taus=None)`
- Plot helper: `plot_adev_with_uncertainties(taus, adev, adev_err)`
- Data type: fractional frequency (`y_k`) sampled every `dt` seconds


In [ ]:
from src.analysis import adev_overlapping_allantools, plot_adev_with_uncertainties
# Reuse 'y' and 'dt' if present; otherwise construct them quickly
try:
    y
    dt
except NameError:
    from src.clocks.ideal import IdealClock
    from src.clocks.noisy import NoisyOscillatorClock
    from src.analysis import run_clocks, fractional_frequency_from_time
    ideal = IdealClock(); noisy = NoisyOscillatorClock(sigma_y=1e-11, seed=7)
    ts = run_clocks([ideal, noisy], duration=12*3600.0, dt=1.0)
    t = ts["time"]; x_noisy = ts["clock_1"]; dt = float(t[1]-t[0])
    y = fractional_frequency_from_time(t, x_noisy, dt=dt)

taus_s, adev, adev_err = adev_overlapping_allantools(y, dt=dt, taus=[1,2,5,10,20,50,100,200,500,1000])
list(zip(taus_s.tolist(), adev.tolist(), adev_err.tolist()))[:5]


In [ ]:
plot_adev_with_uncertainties(taus_s, adev, adev_err)


## Consensus of Three Noisy Clocks (Inverse-Variance)

We demonstrate a **triangular consensus** from three noisy clocks using a simple
**inverse-variance weighting** of their elapsed-time traces. We then compare:

- each noisy clock vs. the **IdealClock**, and  
- the **consensus** vs. the IdealClock.

**Notes**
- This is a **pedagogical** estimator (variance over the full run); in rigorous metrology one would use time-scale methods and stability-aware weights (e.g., Allan/MDEV-based).
- Deterministic seeds ensure reproducibility.


In [ ]:
from src.clocks.ideal import IdealClock
from src.clocks.noisy import NoisyOscillatorClock
from src.clocks.random_walk import RandomWalkFreqClock
from src.clocks.flicker_like import FlickerLikeFreqClock
from src.analysis import run_clocks, compare_clocks, consensus_weighted_average
import numpy as np

# Instantiate clocks (deterministic seeds)
ideal = IdealClock()
c_white = NoisyOscillatorClock(sigma_y=1e-11, seed=11)
c_rw    = RandomWalkFreqClock(sigma_rw=2e-14, seed=22)
c_flick = FlickerLikeFreqClock(sigma_w=5e-12, a=1e-3, seed=33)

duration = 24*3600.0  # 1 day
dt = 1.0

ts = run_clocks([ideal, c_white, c_rw, c_flick], duration=duration, dt=dt)

# Build consensus from the three noisy clocks
keys = ["clock_1", "clock_2", "clock_3"]
cons = consensus_weighted_average(ts, keys, method="inv_var")

# Pairwise comparisons vs. Ideal
metrics = {}
labels = ["WhiteFreq", "RandomWalkFreq", "FlickerLikeFreq"]
for i, lab in zip([1,2,3], labels):
    m = compare_clocks(ts["time"], ts["clock_0"], ts[f"clock_{i}"])
    metrics[lab] = m

# Consensus vs. Ideal
m_cons = compare_clocks(cons["time"], ts["clock_0"], cons["consensus"])

metrics, cons["weights"]


In [ ]:
import matplotlib.pyplot as plt

t = ts["time"]
ideal_series = ts["clock_0"]
white_series = ts["clock_1"]
rw_series    = ts["clock_2"]
flick_series = ts["clock_3"]
cons_series  = cons["consensus"]

# 1) Readings overview
plt.figure()
plt.plot(t, ideal_series, label="Ideal")
plt.plot(t, white_series, label="WhiteFreq")
plt.plot(t, rw_series,    label="RandomWalkFreq")
plt.plot(t, flick_series, label="FlickerLikeFreq")
plt.plot(t, cons_series,  label="Consensus", linewidth=2)
plt.xlabel("Simulation time [s]"); plt.ylabel("Elapsed time reading [s]")
plt.title("Clock readings (Ideal, three noisy, and consensus)")
plt.legend(); plt.grid(True, ls=":")

# 2) Differences vs Ideal
plt.figure()
plt.plot(t, white_series-ideal_series, label="WhiteFreq - Ideal")
plt.plot(t, rw_series-ideal_series,    label="RandomWalkFreq - Ideal")
plt.plot(t, flick_series-ideal_series, label="FlickerLikeFreq - Ideal")
plt.plot(t, cons_series-ideal_series,  label="Consensus - Ideal", linewidth=2)
plt.xlabel("Simulation time [s]"); plt.ylabel("Time difference [s]")
plt.title("Differences relative to Ideal")
plt.legend(); plt.grid(True, ls=":")

# 3) Weights bar chart
plt.figure()
w = cons["weights"]
plt.bar(["White","RW","Flicker"], w)
plt.ylabel("Weight"); plt.title("Consensus weights (sum=1)")
plt.grid(True, axis="y", ls=":")
plt.show()


In [ ]:
# Print metrics concisely
print("=== Pairwise vs Ideal ===")
for lab, m in metrics.items():
    print(f"[{lab}] " + " ".join(f"{k}={v:.3e}" for k, v in m.items()))

print("\n=== Consensus vs Ideal ===")
print(" ".join(f"{k}={v:.3e}" for k, v in m_cons.items()))

print("\nConsensus weights:", [f"{w:.3f}" for w in cons["weights"]])


## Consensus weighting (scalable: any number of noisy clocks)

This section constructs a **consensus** from *all noisy clocks present* in the current run.

- We **exclude** the ideal clock (`clock_0`) from the average.
- Two weighting schemes:
  1. **Fractional-frequency variance** (default): `method = "inv_var_frac"`
  2. **OADEV@τ** (overlapping Allan deviation at a target τ): `method = "inv_oadev_tau"`
- Set `METHOD` and (optionally) `TAU_S` below.
- The code auto-detects available `clock_*` series in `ts` and scales to any count.


In [ ]:
# Ensure project root import for src/
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.clocks.ideal import IdealClock
from src.clocks.noisy import NoisyOscillatorClock
from src.clocks.random_walk import RandomWalkFreqClock
from src.clocks.flicker_like import FlickerLikeFreqClock
from src.analysis import run_clocks, compare_clocks, consensus_weighted_average
import numpy as np

# --- Configure a reproducible run (add/remove clocks freely) ---
ideal = IdealClock()
clocks = [
    ideal,
    NoisyOscillatorClock(sigma_y=1e-11, seed=11),
    RandomWalkFreqClock(sigma_rw=2e-14, seed=22),
    FlickerLikeFreqClock(sigma_w=5e-12, a=1e-3, seed=33),
]

DURATION = 24*3600.0
DT = 1.0
ts = run_clocks(clocks, duration=DURATION, dt=DT)

# Collect all noisy clocks dynamically (exclude clock_0)
noisy_keys = sorted([k for k in ts.keys() if k.startswith("clock_") and k != "clock_0"])

print("Noisy clocks used:", noisy_keys)


In [ ]:
# --- Consensus parameters ---
METHOD = "inv_var_frac"   # choose: "inv_var_frac"  or  "inv_oadev_tau"
TAU_S  = 100.0            # only used if METHOD == "inv_oadev_tau"


In [ ]:
# Build consensus with the chosen method
if METHOD == "inv_oadev_tau":
    cons = consensus_weighted_average(ts, noisy_keys, method=METHOD, dt=DT, tau=TAU_S)
else:
    cons = consensus_weighted_average(ts, noisy_keys, method=METHOD, dt=DT)

weights = cons["weights"]
print("Weights:", [f"{w:.3f}" for w in weights])
if 'var_frac' in cons.get('detail', {}):
    print("Per-clock var_frac:", cons['detail']['var_frac'])
if 'sigma_y_tau' in cons.get('detail', {}):
    print("Per-clock sigma_y_tau:", cons['detail']['sigma_y_tau'], "at tau =", cons['detail'].get('tau'))


In [ ]:
# Compare each noisy clock and the consensus vs. ideal
metrics = {}
for k in noisy_keys:
    m = compare_clocks(ts["time"], ts["clock_0"], ts[k])
    metrics[k] = m

m_cons = compare_clocks(cons["time"], ts["clock_0"], cons["consensus"])

print("=== Pairwise vs Ideal ===")
for k, m in metrics.items():
    print(k, " ".join(f"{kk}={vv:.3e}" for kk, vv in m.items()))

print("
=== Consensus vs Ideal ===")
print(" ".join(f"{kk}={vv:.3e}" for kk, vv in m_cons.items()))


In [ ]:
import matplotlib.pyplot as plt

t = ts["time"]
plt.figure()
plt.plot(t, ts["clock_0"], label="Ideal")
for k in noisy_keys:
    plt.plot(t, ts[k], label=k)
plt.plot(t, cons["consensus"], label="Consensus", linewidth=2)
plt.xlabel("Simulation time [s]"); plt.ylabel("Elapsed time reading [s]")
plt.title(f"Consensus ({METHOD}) over {len(noisy_keys)} noisy clocks")
plt.legend(); plt.grid(True, ls=":")

plt.figure()
for k in noisy_keys:
    plt.plot(t, ts[k]-ts["clock_0"], label=f"{k} - Ideal")
plt.plot(t, cons["consensus"]-ts["clock_0"], label="Consensus - Ideal", linewidth=2)
plt.xlabel("Simulation time [s]"); plt.ylabel("Time difference [s]")
plt.title("Differences relative to Ideal")
plt.legend(); plt.grid(True, ls=":")

plt.figure()
plt.bar(noisy_keys, cons["weights"])
plt.title("Consensus weights (sum=1)")
plt.ylabel("Weight"); plt.grid(True, axis="y", ls=":")
plt.show()
